## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('../Weather_Database/WeatherPy_Database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,69.01,62,53,5.99,broken clouds
1,1,Hermanus,ZA,-34.42,19.23,55.00,85,0,1.99,clear sky
2,2,Abu Samrah,SY,35.30,37.18,86.05,13,78,13.91,broken clouds
3,3,Namibe,AO,-15.20,12.15,70.50,69,0,4.99,clear sky
4,4,Ambovombe,MG,-25.17,46.08,84.02,43,0,7.09,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Abu Samrah,SY,35.30,37.18,86.05,13,78,13.91,broken clouds
4,4,Ambovombe,MG,-25.17,46.08,84.02,43,0,7.09,clear sky
5,5,Bam,IR,29.11,58.36,75.07,19,0,13.76,clear sky
6,6,Vaini,TO,-21.20,-175.20,75.20,88,40,8.05,scattered clouds
7,7,Broken Hill,AU,-31.95,141.43,75.20,25,0,11.41,clear sky
17,17,Vanimo,PG,-2.67,141.30,83.25,75,100,5.77,light rain
21,21,Saint-Philippe,RE,-21.36,55.77,82.00,74,75,10.29,broken clouds
26,26,Kahului,US,20.89,-156.47,80.60,61,1,11.41,clear sky
27,27,Hambantota,LK,6.12,81.12,89.60,66,75,23.04,broken clouds
30,30,Adre,TD,13.47,22.20,84.11,35,0,5.06,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
preferred_cities_df.shape

(230, 10)

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [8]:
clean_df.shape

(229, 10)

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, current description, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Abu Samrah,SY,86.05,broken clouds,35.30,37.18,
4,Ambovombe,MG,84.02,clear sky,-25.17,46.08,
5,Bam,IR,75.07,clear sky,29.11,58.36,
6,Vaini,TO,75.20,scattered clouds,-21.20,-175.20,
7,Broken Hill,AU,75.20,clear sky,-31.95,141.43,
17,Vanimo,PG,83.25,light rain,-2.67,141.30,
21,Saint-Philippe,RE,82.00,broken clouds,-21.36,55.77,
26,Kahului,US,80.60,clear sky,20.89,-156.47,
27,Hambantota,LK,89.60,broken clouds,6.12,81.12,
30,Adre,TD,84.11,clear sky,13.47,22.20,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    response=requests.get(base_url,params=params)
    #print(response.url)
    #commented out the print statement above when adding to github so API key isn't exposed
    hotels=response.json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [12]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Abu Samrah,SY,86.05,broken clouds,35.30,37.18,
4,Ambovombe,MG,84.02,clear sky,-25.17,46.08,MONJA Talilisoa
5,Bam,IR,75.07,clear sky,29.11,58.36,Bam Tourist Hotel
6,Vaini,TO,75.20,scattered clouds,-21.20,-175.20,Keleti Beach Resort
7,Broken Hill,AU,75.20,clear sky,-31.95,141.43,Daydream Motel & Cottages
...,...,...,...,...,...,...,...
691,Kulhudhuffushi,MV,82.74,overcast clouds,6.62,73.07,Haajy Guest House 1
692,Farafenni,GM,75.40,few clouds,13.57,-15.60,Eddy's Hotel
695,Chalmette,US,79.00,overcast clouds,29.94,-89.96,Marina Motel Chalmette
696,Bridgetown,BB,80.60,shower rain,13.10,-59.62,Hilton Barbados Resort


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))